In [4]:
import pandas as pd
import numpy as np
import praw
from praw.models import MoreComments
import os
from groq import Groq

In [5]:
!pip install groq

In [6]:
reddit = praw.Reddit(user_agent="Extract Reddit v1.0 by u/External_Middle8912",
                     client_id="KtFyZUw2ZS0gPLVwnWpITw", 
                     client_secret="8xdnOL6gEBbwCgu389aJrj3c1hKj7w")
# url = "https://www.reddit.com/r/depression/comments/1iswzg5/i_forget_depression_isnt_common/"
category = ["depression", "anxiety", 'mentalillness']
data = []
for item in category:
    subreddit = reddit.subreddit(item)
    for submission in subreddit.hot(limit=1000):
        data.append([
            submission.title,       # Post Title
            submission.selftext,
            submission.score,       # Upvote Score
            submission.num_comments,# Number of Comments
            submission.url,         # Post URL
            submission.author,      # Author Name
            submission.created_utc,  # Timestamp (Unix)
            item
        ])
posts = pd.DataFrame(data,columns=["Title", "Text", "Score", "Comments", "URL", "Author", "Created_UTC", "Subreddit"])       
# indexNames = posts[(posts.body == '[removed]') | (posts.body == '[deleted]')].index
# posts.drop(indexNames, inplace=True)

In [7]:
posts['Text'] = posts['Text'].str.replace(r'\\r\\n|\\n|\\', " ", regex = True)

In [8]:
posts['Text'].iloc[5]

'Whether you’ve tried to end your life but didn’t quite get there or you’ve been in a near death experience, what does death feel like and what happens?\n\nI’m on the brink of suicide. I contemplate it every day. One of the few things that is holding me back is I don’t have any idea of what happens after, or how dying feels.\n\nIf anyone has tried to hang themselves, cut their wrists, or attempted suicide, or even been an accidental near death experience, and has been brought back at the last minute; can you describe the feeling, and what spiritual experience occurs when you get close to fading away?'

In [9]:
posts

,Title,Text,Score,Comments,URL,Author,Created_UTC,Subreddit
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,2364,177,https://www.reddit.com/r/depression/comments/d...,SQLwitch,1.572361e+09,depression
1,"Regular check-in post, with information about ...",Welcome to /r/depression's check-in post - a p...,26,247,https://www.reddit.com/r/depression/comments/1...,SQLwitch,1.727565e+09,depression
2,i regret ever opening up to anyone,they make you feel even shittier about what yo...,74,26,https://www.reddit.com/r/depression/comments/1...,Acrobatic_Lie7392,1.740252e+09,depression
3,I'm going to commit suicide in a few days,I’m 19 years old and I just can't live like th...,52,43,https://www.reddit.com/r/depression/comments/1...,Ewexz,1.740245e+09,depression
4,"I hate happy families, especially dad-daughter...",I feel so disappointed and sad whenever I see ...,41,8,https://www.reddit.com/r/depression/comments/1...,plushielvr,1.740246e+09,depression
...,...,...,...,...,...,...,...,...
2814,Quick Rant.,How the hell is anyone else going to know me i...,1,0,https://www.reddit.com/r/mentalillness/comment...,Consistent_Future159,1.736599e+09,mentalillness
2815,Where do you start?,I've been struggling with mental health since ...,1,1,https://www.reddit.com/r/mentalillness/comment...,MsKonduct,1.736596e+09,mentalillness
2816,Whats wrong with me,This is not a cry for help I genuinely just wa...,4,0,https://www.reddit.com/r/mentalillness/comment...,ButterflyOk7064,1.736566e+09,mentalillness
2817,planning relapses,I just realized that I've been planning my rel...,3,1,https://www.reddit.com/r/mentalillness/comment...,Miserable_Tower_6924,1.736567e+09,mentalillness


In [27]:
groq_API = "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"
client = Groq(api_key=groq_API)

def categorize_reddit_content(content):
      messages = [
          {
              "role": "system",
              "content": 'null'
          },
          {
              "role": "user",
              "content": f"Please categorize our reddit post content into one of the three categories: 'Depression', 'Anxiety', 'Mental Illness': \n\n {content}. The output should only be a word with no extra texts."
          }
      ]

      # prompt = "Please categorize our reddit post content into one of the three categories: 'Depression', 'Anxiety', 'Mental Illness'."

      chat_completion = client.chat.completions.create(
              temperature = 0.15, 
              messages = messages, 
              model = "llama3-70b-8192", 
              n = 1, 
      )
      category = chat_completion.choices[0].message.content
      return category

# print(chat_completion.choices[0].message.content)

In [23]:
sub_df = posts
sub_df["New_Text"] = sub_df["Text"].apply(categorize_reddit_content)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jmqkkgdvfeds21h90bdrzsqd` service tier `on_demand` on : Limit 500000, Used 499983, Requested 891. Please try again in 2m30.9542s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': '', 'code': 'rate_limit_exceeded'}}

In [26]:
sub_df

,Title,Text,Score,Comments,URL,Author,Created_UTC,Subreddit
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,2364,177,https://www.reddit.com/r/depression/comments/d...,SQLwitch,1.572361e+09,depression
1,"Regular check-in post, with information about ...",Welcome to /r/depression's check-in post - a p...,26,247,https://www.reddit.com/r/depression/comments/1...,SQLwitch,1.727565e+09,depression
2,i regret ever opening up to anyone,they make you feel even shittier about what yo...,74,26,https://www.reddit.com/r/depression/comments/1...,Acrobatic_Lie7392,1.740252e+09,depression
3,I'm going to commit suicide in a few days,I’m 19 years old and I just can't live like th...,52,43,https://www.reddit.com/r/depression/comments/1...,Ewexz,1.740245e+09,depression
4,"I hate happy families, especially dad-daughter...",I feel so disappointed and sad whenever I see ...,41,8,https://www.reddit.com/r/depression/comments/1...,plushielvr,1.740246e+09,depression
...,...,...,...,...,...,...,...,...
2814,Quick Rant.,How the hell is anyone else going to know me i...,1,0,https://www.reddit.com/r/mentalillness/comment...,Consistent_Future159,1.736599e+09,mentalillness
2815,Where do you start?,I've been struggling with mental health since ...,1,1,https://www.reddit.com/r/mentalillness/comment...,MsKonduct,1.736596e+09,mentalillness
2816,Whats wrong with me,This is not a cry for help I genuinely just wa...,4,0,https://www.reddit.com/r/mentalillness/comment...,ButterflyOk7064,1.736566e+09,mentalillness
2817,planning relapses,I just realized that I've been planning my rel...,3,1,https://www.reddit.com/r/mentalillness/comment...,Miserable_Tower_6924,1.736567e+09,mentalillness


In [ ]:
# Call this function in the notebook or when using the API
if __name__ == "__main__":
    # Test categorization with a sample post (or input from the frontend)
    category = categorize_reddit_content(input)
    print(category)  # You can format this for JSON or a suitable response
